In [1]:
import fileIO
import splice_files
import validate_timestamps
import coordinate_utilities
import time_utilities

In [6]:
# enter name and directory of input file
# input_filename1  = 'ouutDir/fixed.gpx'
input_filename1  = 'workingDir/Evening_Run-3.gpx'
input_filename2  = 'outputDir/garbagio.gpx'

output_filename1 = 'outputDir/fixed.gpx'

# select data fields to import
data_flags1 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  True, 
    'pwrflag':  False 
}
data_flags2 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}

In [7]:
trkptlist1, meta_data1 = fileIO.import_trackpoints(data_flags1, input_filename1)
trkptlist2, meta_data2 = fileIO.import_trackpoints(data_flags2, input_filename2)

4709 points imported from workingDir/Evening_Run-3.gpx.
401 points imported from outputDir/garbagio.gpx.


Generate timestamps for the missing GPX coordinates

In [8]:
gap_times = coordinate_utilities.find_gap(trkptlist=trkptlist1, mode=1)
print(f"{input_filename1} gap: {gap_times}")

workingDir/Evening_Run-3.gpx gap: ('2024-01-28T20:42:29Z', '2024-01-28T20:49:50Z')


Check if the timestamps in my gpx log got messed up from the change in timezone:

In [9]:
validate_timestamps.compare_trkpt(trkptlist1,verbose=False)

Fail: 0, Success: 4707


Now we want to identify the gap between Chewbacca rock and the Lottery Farm. This involves finding the index in trkptlist1 where the watch was paused and then unpaused.

In [10]:
gap_times = coordinate_utilities.find_gap(trkptlist=trkptlist1, mode=1)
print(f"{input_filename1} gap: {gap_times}")
out1 = coordinate_utilities.find_gap(trkptlist=trkptlist1, mode=2)
print(f"{input_filename1} gap: {out1}")

workingDir/Evening_Run-3.gpx gap: ('2024-01-28T20:42:29Z', '2024-01-28T20:49:50Z')
workingDir/Evening_Run-3.gpx gap: ((51.483538, 5.652798), (51.481579, 5.654472))


Now find the point in trkptlist2 which is the closest to where the watch was unpaused (i.e. the unpausing timestamp in trkptlist1)

In [11]:
ts = {'year': 2024, 'month': 1, 'day': 28, 'hour': 20, 'minute': 49, 'second': 50}
index = splice_files.get_trackpoint_index_by_time(trkptlist1, ts) # this is the index of the gap of the lottery farm
print("Index of unpausing in trkptlist1: ",index)

Index of unpausing in trkptlist1:  3954


In [12]:
# now find the point in trkptlist2 closest to the lottery farm
out = coordinate_utilities.find_closest(trkptlist1[index],trkptlist2)

trkptlist1_index = index # chebacca rock summit
trkptlist2_index_start = 0 # chebacca rock summit
trkptlist2_index_finish = out-1 #lottery farm

Finding closest point to:  (51.481579,5.654472)...
Closest point in the list: (51.47958,5.65605) at index=400


Splicing the files:

In [13]:
trkptlist3 = splice_files.fix_pause_by_index(trkptlist1, trkptlist2, trkptlist1_index, trkptlist2_index_start, trkptlist2_index_finish)

Export the file:

In [14]:
data_flags3 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
fileIO.export_trackpoints(data_flags3, trkptlist3, output_filename1)

Exporting file...
File exported to outputDir/fixed.gpx
